In [20]:
%%configure -f
{"executorCores": 4, "executorMemory": "8096M", "conf": {"spark.default.parallelism": 1000}}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1573663397043_0003,pyspark,idle,,,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1573663397043_0003,pyspark,idle,Link,Link,✔


In [21]:
import pandas as pd

import gensim
import math
from gensim.utils import tokenize
from gensim.parsing.preprocessing import remove_stopwords
import string
from string import punctuation
import numpy as np
from langdetect import detect
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, IntegerType, ArrayType, DoubleType, MapType
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, StopWordsRemover, CountVectorizer
from pyspark.ml import Pipeline
import spacy

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
df = spark.read.load("s3://ai-data-lake-dev-eu-west-1/business/company_data_denormalized")
# df = df.distinct()
#df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
df_orbis = df.filter(F.col('vendor')=='CapitalIQ').cache()
df_capiq = df.filter(F.col('vendor')=='Orbis').cache()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
df_orbis.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

7993468

In [25]:
df_capiq.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

3106630

In [26]:
def process_text(text):
    if text:
        return [x.strip(punctuation) for x in text.lower().split()]
    else:
        return ""

process_text_udf = udf(process_text, ArrayType(StringType()))

def calcIdf(doc_count, df):
    return math.log((doc_count+1)/(df+1))

calIdf_udf = udf(calcIdf, DoubleType())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
df_preprocess = (df.withColumn("tokenized_text", process_text_udf("short_description"))
                 .withColumn("idx", F.monotonically_increasing_id())
                 .withColumn('token', F.explode('tokenized_text'))
                )

df_tokens = (df_preprocess.groupBy("token")
             .agg(F.countDistinct("idx").alias('df'))
            )

nb_docs = df_tokens.count()

df_idf = df_tokens.withColumn("idf", calIdf_udf(F.lit(nb_docs), 'df')).select('token', 'idf').sort(F.col("idf"))

idf = df_idf.toPandas()

idf_dict = dict(zip(idf.token, idf.idf))

print(len(idf_dict))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1469374

In [28]:
df_orbis_preprocess = (df_orbis.withColumn("tokenized_text", process_text_udf("short_description"))
                 .withColumn("idx", F.monotonically_increasing_id())
                 .withColumn('token', F.explode('tokenized_text'))
                )

df_orbis_tokens = (df_orbis_preprocess.groupBy("token")
             .agg(F.countDistinct("idx").alias('df'))
            )

nb_docs = df_orbis_tokens.count()

df_orbis_idf = df_orbis_tokens.withColumn("idf", calIdf_udf(F.lit(nb_docs), 'df')).select('token', 'idf').sort(F.col("idf"))

orbis_idf = df_orbis_idf.toPandas()

orbis_dict = dict(zip(orbis_idf.token, orbis_idf.idf))

print(len(orbis_dict))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1400564

In [29]:
orbis_dict['brand']

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

4.1355700467578815

In [30]:
df_capiq_preprocess = (df_capiq.withColumn("tokenized_text", process_text_udf("short_description"))
                 .withColumn("idx", F.monotonically_increasing_id())
                 .withColumn('token', F.explode('tokenized_text'))
                )

df_capiq_tokens = (df_capiq_preprocess.groupBy("token")
             .agg(F.countDistinct("idx").alias('df'))
            )

nb_docs = df_capiq_tokens.count()

df_capiq_idf = df_capiq_tokens.withColumn("idf", calIdf_udf(F.lit(nb_docs), 'df')).select('token', 'idf').sort(F.col("idf"))

capiq_idf = df_capiq_idf.toPandas()
capiq_dict = dict(zip(capiq_idf.token, capiq_idf.idf))

print(len(capiq_dict))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

131950

In [31]:
capiq_dict['brand']

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

4.330271154289916

In [32]:
diff_word = []
diff_idf = []
for word_orbis, idf_orbis in orbis_dict.items():
    if word_orbis in idf_dict:
        diff_word.append(word_orbis)
        diff_idf.append(abs(idf_orbis - idf_dict[word_orbis]))
#         if abs(idf_cap - orbis_dict[word_cap]) > 4:
#             print(word_cap)

idx = sorted(range(len(diff_idf)), key=lambda k: diff_idf[k], reverse=True)
diff_word = [diff_word[x] for x in idx]
diff_id = [diff_idf[x] for x in idx]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
diff_word[0:1000]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['dijk', 'installatio', 'stipulated', 'n.e.c', 'wor', 'self-run', 'stipulations', 'mimeographing', "subject's", 'subdividing', 'excluding', 'excl', 'shall', 'bureau', 'produ', 'suport', "state's", 'non-specialized', 'prohibited', 'steel-frame', 'fabri', 'predominating', 'non-specialised', 'laminboard', 'polywood', 'equipmen', 'thiss', 'plaiting', 'constru', 'unclassified', 'self-produced', 'grouped', 'non-cast', 'made-up', 'junkets', 'forbidden', 'van', 'monopolized', 'manufact', 'chss', 'papier-mache', 'source', 'doanh', 'above-mentioned', 'termek', 'discretion', 'food-stuffs', 'chuy', 'elelmiszer', 'bumboats', 'trad', 'trong', 'video-tape', 'genaral', 'bank/financial', 'imitative', 'low-alcoholic', 'cons', 'accessaries', 'vechicles', 'manufac', 'roll-forming', 'nondepository', 'crocheted', 'ugynoki', 'prohibit', 'termekek', 'light-vehicle', 'laws', 'paper-producing', 'berendezesek', 'giss', 'actvities', 'telecommunication-related', 'handphones', 'subject', '3,5', '9001', 'elsewhere',

In [34]:
diff_word = []
diff_idf = []
for word_cap, idf_cap in capiq_dict.items():
    if word_cap in orbis_dict:
        diff_word.append(word_cap)
        diff_idf.append(abs(idf_cap - orbis_dict[word_cap]))
#         if abs(idf_cap - orbis_dict[word_cap]) > 4:
#             print(word_cap)

idx = sorted(range(len(diff_idf)), key=lambda k: diff_idf[k], reverse=True)
diff_word = [diff_word[x] for x in idx]
diff_id = [diff_idf[x] for x in idx]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
diff_word[0:100]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['dijk', 'nsk', 'installatio', 'stipulated', 'n.e.c', 'wor', 'self-run', 'stipulations', 'mimeographing', "subject's", 'subdividing', 'excluding', 'excl', 'shall', 'bureau', 'produ', 'suport', "state's", 'non-specialized', 'prohibited', 'steel-frame', 'fabri', 'predominating', 'non-specialised', 'laminboard', 'polywood', 'equipmen', 'thiss', 'plaiting', 'constru', 'unclassified', 'self-produced', 'grouped', 'non-cast', 'made-up', 'junkets', 'forbidden', 'van', 'monopolized', 'manufact', 'chss', 'papier-mache', 'source', 'doanh', 'above-mentioned', 'termek', 'discretion', 'food-stuffs', 'chuy', 'elelmiszer', 'bumboats', 'trad', 'trong', 'video-tape', 'genaral', 'bank/financial', 'imitative', 'tokyo', 'low-alcoholic', 'vechicles', 'manufac', 'cons', 'accessaries', 'nondepository', 'roll-forming', 'ugynoki', 'crocheted', 'prohibit', 'nec', 'termekek', 'massachusetts', 'formerly', 's.l', 'light-vehicle', 'laws', 'paper-producing', 'pty', 'berendezesek', 'california', 'toronto', 'giss', 'ac

In [36]:
from pyspark.ml.feature import NGram
df_orbis = df_orbis.withColumn("short_description_tokens", F.split("short_description", " "))
ngram = NGram(n=2, inputCol="short_description_tokens", outputCol="bigrams")
df_orbis_ngram = ngram.transform(df_orbis)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
df_orbis_ngram_count = (
  df_orbis_ngram.withColumn("bigrams_explode", F.explode(F.col("bigrams")))
  .groupBy("bigrams_explode")
  .agg(F.count("*").alias('count'))
  .sort(F.col("count").desc())
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
# df_orbis_ngram_count.where(df_orbis_ngram_count.bigrams_explode == "brand name").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [65]:
word = 'operator'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [66]:
orbis_dict[word]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

5.715319137664061

In [67]:
capiq_dict[word]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2.8086299797591336